# OData

## Extract

In [1]:
import requests
import pandas as pd
import math


def read_odata(url):
    # Sending a GET request to the OData service
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful

    # Converting the response to a pandas DataFrame
    data = response.json()
    df = pd.json_normalize(data['value'])
    return df

def count_odata(base_url):
    url = base_url + '/$count'
    # Sending a GET request to the OData service
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    columns = response.headers['Content-Length']
    rows = response.json()
    return columns, rows

def auto_read(base_url, rows):
    if rows >= 5000:
        n = math.ceil(rows / 5000)
        df_list = []
        for x in range(n):
            skip = f"&$skip={x*5000}"
            url = f"{base_url}?$top=5000{skip}"
            df_list.append(read_odata(url))
        return pd.concat(df_list, ignore_index=True)
    else:
        return read_odata(base_url)

def read_all(source):
    # Get primary identification
    primary_df = read_odata(source)

    # Initialize a dictionary to store the data frames
    tables_dict = {}
    
    # Iterate over each row in primary_df
    for index, table in primary_df.iterrows():
        print(f"\tTable {index}: {table['name']}")
        columns, rows = count_odata(table['url'])
        print(f"\tColumns: {columns}  Rows: {rows}\n")
        # Create a DataFrame per table and store it in the dictionary
        tables_dict[table['name']] = auto_read(table['url'], rows)

    return tables_dict


# "Primaire identificatie" te vinden op https://data.overheid.nl/dataset/443-bestaande-koopwoningen--verkoopprijzen-prijsindex-2015-100#panel-description
source = 'https://opendata.cbs.nl/ODataApi/OData/83906NED'

print("Schema info:\n")
tables_dict = read_all(source)
print("Main DataSet:")
tables_dict["TypedDataSet"].head()


Schema info:

	Table 0: TableInfos
	Columns: 1  Rows: 1

	Table 1: UntypedDataSet
	Columns: 3  Rows: 493

	Table 2: TypedDataSet
	Columns: 3  Rows: 493

	Table 3: DataProperties
	Columns: 2  Rows: 11

	Table 4: CategoryGroups
	Columns: 1  Rows: 0

	Table 5: Perioden
	Columns: 3  Rows: 493

Main DataSet:


,ID,Perioden,PrijsindexBestaandeKoopwoningen_1,OntwikkelingTOVVoorgaandePeriode_2,OntwikkelingTOVEenJaarEerder_3,AantalVerkochteWoningen_4,OntwikkelingTOVVoorgaandePeriode_5,OntwikkelingTOVEenJaarEerder_6,GemiddeldeVerkoopprijs_7,TotaleWaardeVerkoopprijzen_8
0,0,1995MM01,41.3,NaN,NaN,8793,NaN,NaN,88991,782
1,1,1995MM02,41.5,0.6,NaN,10103,14.9,NaN,89741,907
2,2,1995MM03,41.4,-0.3,NaN,11838,17.2,NaN,90432,1071
3,3,1995KW01,41.4,NaN,NaN,30734,NaN,NaN,89792,2760
4,4,1995MM04,41.6,0.5,NaN,10626,-10.2,NaN,89808,954
